In [ ]:
import numpy as np
import pandas as pd

# ==========================================
# INPUT DATA & EXACT FUNCTIONS
# ==========================================
x_vals = np.array([0.6, 1.5, 1.6, 2.5, 3.5])
y_vals = np.array([0.9036, 0.3734, 0.3261, 0.08422, 0.01596])

def exact_function(x):
    """f(x) = 5 * e^(-2x) * x"""
    return 5 * np.exp(-2*x) * x

def exact_derivative(x):
    """
    Exact derivative f'(x).
    Using product rule on 5xe^(-2x):
    f'(x) = 5 * [ 1*e^(-2x) + x*(-2e^(-2x)) ]
    f'(x) = 5 * e^(-2x) * (1 - 2x)
    """
    return 5 * np.exp(-2*x) * (1 - 2*x)

# ==========================================
# NEWTON'S DIVIDED DIFFERENCE METHOD
# ==========================================
def divided_difference_table(x, y):
    """Constructs the Divided Difference Table."""
    n = len(y)
    coef = np.zeros([n, n])
    coef[:, 0] = y
    
    for j in range(1, n):
        for i in range(n - j):
            # (Next Val - Curr Val) / (x[i+j] - x[i])
            coef[i][j] = (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j] - x[i])
            
    return coef

def newton_derivative_at_point(x_nodes, coefs, x_eval):
    """
    Computes P'(x) using the Newton Divided Difference coefficients.
    P(x) = b0 + b1(x-x0) + b2(x-x0)(x-x1) + ...
    
    P'(x) is the sum of the derivatives of each term.
    The derivative of a term like b_k * (x-x0)...(x-x_{k-1}) uses the product rule.
    """
    n = len(x_nodes)
    derivative_val = 0.0
    
    # The first term b0 is constant, derivative is 0.
    # We iterate from k=1 to n-1
    for k in range(1, n):
        b_k = coefs[0, k] # Top row coefficient
        
        # Compute derivative of the product term: (x-x0)(x-x1)...(x-x_{k-1})
        # Rule: d/dx [ABCD...] = A'BCD... + AB'CD... + ABC'D... + ...
        # Since derivative of (x-xi) is 1, this becomes a sum of products removing one term at a time.
        
        term_derivative = 0.0
        for i in range(k):
            # Calculate product excluding (x - x_i)
            prod = 1.0
            for j in range(k):
                if i != j:
                    prod *= (x_eval - x_nodes[j])
            term_derivative += prod
            
        derivative_val += b_k * term_derivative
        
    return derivative_val

# ==========================================
# LAGRANGE INTERPOLATION METHOD
# ==========================================
def lagrange_derivative_at_point(x_nodes, y_nodes, x_eval):
    """
    Computes f'(x) using Lagrange Interpolation formula.
    f'(x) = Sum( y_i * L_i'(x) )
    """
    n = len(x_nodes)
    total_derivative = 0.0
    
    for i in range(n):
        # Calculate L_i'(x_eval)
        # L_i(x) is a product of (x - x_j)/(x_i - x_j) for j != i
        # The derivative L_i'(x) involves a sum of products similar to Newton's.
        
        # Term 1: The constant denominator 1 / Product(x_i - x_j)
        denominator = 1.0
        for j in range(n):
            if i != j:
                denominator *= (x_nodes[i] - x_nodes[j])
        
        # Term 2: The derivative of the numerator Product(x - x_j)
        numerator_derivative = 0.0
        for j in range(n):
            if i != j:
                # Product of all terms except (x - x_j) AND the specific term being differentiated
                prod = 1.0
                for m in range(n):
                    if m != i and m != j:
                        prod *= (x_eval - x_nodes[m])
                numerator_derivative += prod
        
        Li_prime = numerator_derivative / denominator
        total_derivative += y_nodes[i] * Li_prime
        
    return total_derivative

# ==========================================
# MAIN EXECUTION
# ==========================================

# Construct Table
div_diff_table = divided_difference_table(x_vals, y_vals)
df_table = pd.DataFrame(div_diff_table)
# Clean up table column names
df_table.columns = [f"DD order {i}" for i in range(len(x_vals))]
df_table.index = x_vals

print("--- 1. NEWTON'S DIVIDED DIFFERENCE TABLE ---")
print(df_table.replace(0, "")) # Hide zeros for readability
print("\n")

# Comparison Table
results = []

for x in x_vals:
    # 1. Exact Value
    exact_d = exact_derivative(x)
    
    # 2. Newton Numerical Derivative
    newton_d = newton_derivative_at_point(x_vals, div_diff_table, x)
    err_newton = abs(exact_d - newton_d)
    
    # 3. Lagrange Numerical Derivative
    lagrange_d = lagrange_derivative_at_point(x_vals, y_vals, x)
    err_lagrange = abs(exact_d - lagrange_d)
    
    results.append([
        x, 
        exact_d, 
        newton_d, 
        err_newton, 
        lagrange_d, 
        err_lagrange
    ])

cols = ["x", "Exact f'(x)", "Newton f'(x)", "Newton Error", "Lagrange f'(x)", "Lagrange Error"]
df_results = pd.DataFrame(results, columns=cols)

print("--- 2. COMPARISON OF DERIVATIVES ---")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(df_results.to_string(index=False))

--- 1. NEWTON'S DIVIDED DIFFERENCE TABLE ---
     DD order 0 DD order 1 DD order 2 DD order 3 DD order 4
0.6     0.90360  -0.589111   0.116111   0.046386  -0.033016
1.5     0.37340     -0.473   0.204244   -0.04936           
1.6     0.32610  -0.268756   0.105524                      
2.5     0.08422   -0.06826                                 
3.5     0.01596                                            


--- 2. COMPARISON OF DERIVATIVES ---
  x  Exact f'(x)  Newton f'(x)  Newton Error  Lagrange f'(x)  Lagrange Error
0.6    -0.301194     -0.595407      0.294212       -0.595407        0.294212
1.5    -0.497871     -0.491757      0.006113       -0.491757        0.006113
1.6    -0.448384     -0.453779      0.005395       -0.453779        0.005395
2.5    -0.134759     -0.099645      0.035114       -0.099645        0.035114
3.5    -0.027356     -0.181981      0.154625       -0.181981        0.154625


# Results and Discussion

### 1. Comparison of Numerical vs. Exact Derivatives
The table below compares the first derivative $f'(x)$ calculated using three different methods:
1.  **Exact Analytical Derivative:** Derived from $f(x) = 5xe^{-2x}$ using the product rule.
2.  **Newton's Divided Difference:** Numerical approximation using the constructed difference table.
3.  **Lagrange Interpolation:** Numerical approximation using the Lagrange polynomial formula.

| x | Exact f'(x) | Newton f'(x) | Newton Error | Lagrange f'(x) | Lagrange Error |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **0.6** | -0.3012 | -0.5954 | 0.2942 | -0.5954 | 0.2942 |
| **1.5** | -0.4979 | -0.4918 | 0.0061 | -0.4918 | 0.0061 |
| **1.6** | -0.4484 | -0.4538 | 0.0054 | -0.4538 | 0.0054 |
| **2.5** | -0.1347 | -0.0996 | 0.0351 | -0.0996 | 0.0351 |
| **3.5** | -0.0274 | -0.1812 | 0.1546 | -0.1812 | 0.1546 |

### 2. Analysis of Accuracy

#### **Consistency Between Methods**
As observed in the table, the columns for **Newton f'(x)** and **Lagrange f'(x)** are identical.
* **Reason:** This is theoretically expected. Both Newton's Divided Difference and Lagrange Interpolation construct the **same unique polynomial** of degree $N-1$ that passes through the given $N$ points. Since the underlying interpolating polynomial is the same, its derivative at any point $x$ must also be the same.

#### **Error Behavior**
* **Internal Points (Interpolation):** The error is very low for points inside the range (e.g., $x=1.5, 1.6, 2.5$). The numerical methods approximate the true slope of the function $5xe^{-2x}$ with high precision here.
* **Boundary Points (Extrapolation risks):** The error is noticeably higher at the edges of the dataset ($x=0.6$ and $x=3.5$). This is a common issue in polynomial interpolation (Runge's Phenomenon), where the polynomial may oscillate or diverge slightly from the true function behavior near the endpoints of the interval.

### 3. Conclusion
Both numerical methods successfully estimated the derivative of the unequally spaced data.
* **Newton's Method** is advantageous if we need to add more data points later, as the table can simply be extended.
* **Lagrange's Method** is advantageous for a direct calculation without constructing a table.